In [1]:
import MySQLdb
import matplotlib
matplotlib.use ('template')
import matplotlib.pyplot as plt
import datetime
import numpy as np
import pandas as pd
import pylab as pl
import os
import matplotlib.colors as mcolors

In [2]:
nivel = [92,106,108,115,96,101,116,134,182,109,155,161,166,169,152,104,179,90,98,238,173]
precipitacion = [(1,8,29),(33,58,57,171),(18,3,25),(52,64,15),(20,48),(5,56,26,49),(54,9,17,29),(89,48,14,121),
                 (5,46,129,44,42),(64,65,34),(37,14,21,51),(38,62,36),(8,29,1),(57,58,171),(9,29,17),(15,56),
                 (57,33,61),(3,43,18,25),(17,27,11),(11,27,21,20), (29,8,1)]
tipos = ['Ni','Ni','Ni','Ni','Ni','pr','pr','pr','Ni','Ni','pr','ni','pr','pr','pr','pr','ni','pr','pr','pr','ni']
cuencas = ['Altavista','Tres_aguas','Santa_rita','La_Ayura','La_Gomez','La_Presidenta','La_Picacha','La_Madera',
          'Santa_Elena','La_Doctora','El_Hato','La_Chocha','El_Chocho','La_Clara','Ana_Diaz','La_Zuniga','La_Inmaculada',
          'El_Encanto','La_Hueso','La_Iguana','Aguas_Frias']

### Parametros estacion a actualizar

In [3]:
n = 173
ee = np.where(np.array(nivel) == n)[0]
p = np.array(precipitacion)[ee]
tip = np.array(tipos)[ee]
umb_niv = 60
path='/media/nicolas/Home/Jupyter/Esneider/modelo_crecidas/Buscador_eventos/'+np.array(cuencas)[ee][0]+'/'
ctec=['2030-09-12 15:23','2030-09-17 14:07','2030-09-12 13:25']
#ctec = ['2020-09-17 13:01','2013-09-09 15:51','2020-09-17 13:01'] La Inmaculada

FI='2017-06-14'
FF='2018-01-18'

### Find maximos

In [4]:
# Consulta en base de datos

host      ='192.168.1.74'
user      ='usrCalidad'
passw     ='aF05wnXC;'
bd_nombre ='siata'

Estaciones="SELECT Codigo,Nombreestacion, offsetN, red  FROM estaciones WHERE codigo=("+str(n)+")"
db = MySQLdb.connect(host, user,passw,bd_nombre)
db_cursor = db.cursor()
db_cursor.execute(Estaciones)
Cod = db_cursor.fetchall()
date_ini=datetime.datetime(int(FI[0:4]),int(FI[5:7]),int(FI[8:10]))
date_fin=datetime.datetime(int(FF[0:4]),int(FF[5:7]),int(FF[8:10]))
sql_datos ="SELECT fecha, DATE_FORMAT(fecha,'%Y-%m-%d'), hora, DATE_FORMAT(hora, '%H:%i:%s'), Cliente, (" +str(Cod[0][2])+"-"+tip[0]+"), calidad FROM datos WHERE cliente = "+str(n)+" and (((fecha>'"+str(date_ini.year)+"-"+str(date_ini.month)+"-"+str(date_ini.day)+"') or (fecha='"+str(date_ini.year)+"-"+str(date_ini.month)+"-"+str(date_ini.day)+"' and hora>='"+str(date_ini.hour)+":"+str(date_ini.minute)+":00')) and ((fecha<'"+str(date_fin.year)+"-"+str(date_fin.month)+"-"+str(date_fin.day)+"') or (fecha='"+str(date_fin.year)+"-"+str(date_fin.month)+"-"+str(date_fin.day)+"' and hora<='"+str(date_fin.hour)+":"+str(date_fin.minute)+":00')))"

db = MySQLdb.connect(host, user,passw,bd_nombre)
db_cursor = db.cursor()
db_cursor.execute(sql_datos)
data = db_cursor.fetchall()

db_cursor.execute(Estaciones)
Nombre = db_cursor.fetchall()

In [5]:
F=[]
H=[]
ID=[]
N=[]
Calidad=[]
FH=[]

for dato in data:
    if dato[6] != 151:
        F.append(dato[1])
        H.append(dato[3])
        ID.append(dato[4])
        N.append(dato[5])
        Calidad.append(dato[6])  
    
for i in range(len(F)):
    fh=F[i]+" "+H[i]
    FH.append(fh)
    
#Hasta este punto se han extraido datos de nivel y tal cual estan en la base de datos
#vector fechas para comparar fechas de las estaciones de Nivel y Precipitacion
    
EjeX=[]
EjeY=[]
formato = '%Y-%m-%d %H:%M:%S'
C=0

ID2=[]
Calidad2=[]
FH1 = []
        
vv = np.where(np.array(Calidad) != 151)[0]
EjeY = np.array(N)[vv]
FH1 = np.array(FH)[vv]
    
for i in FH1:
    EjeX.append(datetime.datetime.strptime (i[:-3], '%Y-%m-%d %H:%M'))        

D = pd.Series(EjeY, EjeX)

#Suavizado de serie
j=0
while j<2:
    d_mva = pd.rolling_mean(D,20,min_periods=8,center=True)
    D1=d_mva
    j+=1

#Hallar maximos
Ejey = D1.values
EjeX = D1.index

ite = 3
umbral = umb_niv
busq = 30

#Hallar maximos locales

def maximos(Ejy,EjX):
    
    grad = []
    maxi = []
    picos = []
    datesM = []
    
    for j in range(len(Ejy)-1):
        if Ejy[j+1] == Ejy[j]:
            grad.append(1)
        
        if Ejy[j+1] != Ejy[j]:
            grad.append((Ejy[j+1]-Ejy[j])/abs(Ejy[j+1]-Ejy[j]))     
    
    for i in range(len(grad)-1):
        maxi.append(grad[i+1]-grad[i])
         
    pp = (np.where(np.array(maxi) == -2))[0]
    pp = pp+np.ones(len(pp))
    
    for x in range(len(pp)):
        picos.append(Ejy[int(pp[x])])
        datesM.append(EjX[int(pp[x])])

    return picos,datesM
    
for i in range(ite):
    if i == 0:
        aux = maximos(Ejey,EjeX)
    if i > 0:
        aux = maximos(aux[0],aux[1])
        
pi1 = []
da1 = []
    
for y in aux[1]:
    fa = np.where(EjeX == y)[0]
    fa = fa[len(fa)-1]
    if fa > busq:
        ma = np.where(EjeY[fa-busq:fa] == np.max(EjeY[fa-busq:fa]))[0]
        pi1.append(EjeY[fa-(busq-ma[len(ma)-1])])
        da1.append(EjeX[fa-(busq-ma[len(ma)-1])])     

dM = np.array(da1)
NM = np.array(pi1) 
dM = dM[NM >= umbral]
NM = NM[NM >= umbral]
M = pd.Series(NM, dM)

fig1=pl.figure(1,edgecolor='w',facecolor='w',figsize=(15,15))
plt.close('all')
plt.clf ()
plt.cla ()
D.plot()
D1.plot()
M.plot()
plt.savefig(path+'maximos.png')

np.savetxt(path+'fechas_1.txt',np.array(dM),fmt="%s")
np.savetxt(path+'niveles_1.txt',NM)

print '--------------------------------------------------------------------------'
print 'Graficar las hidrografas de los eventos nuevos y seleccionar los buenos'
print '--------------------------------------------------------------------------'


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:44: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(min_periods=8,window=20,center=True).mean()


--------------------------------------------------------------------------
Graficar las hidrografas de los eventos nuevos y seleccionar los buenos
--------------------------------------------------------------------------


### Graficar eventos

In [6]:
fechas=np.loadtxt(path+'fechas_1.txt',dtype=str)

# open database connection
host      ='192.168.1.74'
user      ='usrCalidad'
passw     ='aF05wnXC;'
bd_nombre ='siata'

Estaciones="SELECT Codigo,Nombreestacion, offsetN, red  FROM estaciones WHERE codigo=("+str(n)+")"
db = MySQLdb.connect(host, user,passw,bd_nombre)
db_cursor = db.cursor()
db_cursor.execute(Estaciones)
Cod = db_cursor.fetchall()

In [7]:
# Itera por evento

e = p[0]

for w in np.arange(0,len(fechas),1):
    i=fechas[w]
    print i
    dt_fecha = datetime.datetime(int(i[0][0:4]),int(i[0][5:7]),int(i[0][8:10]),int(i[1][0:2]),int(i[1][3:5]))

    #extraer de la base de datos fecha, hora y nivel el vector fecha debe quedar en el formato '%Y-%m-%d %H:%M:%S' datetime
    date_ini=dt_fecha-datetime.timedelta(minutes=60)
    date_fin=dt_fecha+datetime.timedelta(minutes=180)
    datos_n = "SELECT fecha, DATE_FORMAT(fecha,'%Y-%m-%d'), hora, DATE_FORMAT(hora, '%H:%i:%s'), (" +str(Cod[0][2])+"-"+tip[0]+"), calidad FROM datos WHERE cliente = "+str(n)+" and (((fecha>'"+str(date_ini.year)+"-"+str(date_ini.month)+"-"+str(date_ini.day)+"') or (fecha='"+str(date_ini.year)+"-"+str(date_ini.month)+"-"+str(date_ini.day)+"' and hora>='"+str(date_ini.hour)+":"+str(date_ini.minute)+":00')) and ((fecha<'"+str(date_fin.year)+"-"+str(date_fin.month)+"-"+str(date_fin.day)+"') or (fecha='"+str(date_fin.year)+"-"+str(date_fin.month)+"-"+str(date_fin.day)+"' and hora<='"+str(date_fin.hour)+":"+str(date_fin.minute)+":00')))"
    db = MySQLdb.connect(host, user,passw,bd_nombre)
    db_cursor = db.cursor()
    db_cursor.execute(datos_n)
    data = db_cursor.fetchall()

    FN=[]
    N=[]
    calidad=[]

    for k in range(len(data)):
        try:
            dato = data[k]
            dt = datetime.datetime(int(dato[1][0:4]),int(dato[1][5:7]),int(dato[1][8:10]),int(dato[3][0:2]),int(dato[3][3:5]))
            FN.append(dt)
            N.append(dato[4])
            calidad.append(dato[5])
            if dt == dt_fecha:
                pp = k
        except:
            pass

    cc = np.where(np.array(calidad) != 151)[0]
    FN = np.array(FN)[cc]
    N = np.array(N)[cc]

    N=pd.Series(map(float,N),FN) # construir serie de nivel en formato pandas

    for j in e:
        date_ini=dt_fecha-datetime.timedelta(minutes=60)
        date_fin=dt_fecha+datetime.timedelta(minutes=30)
        datos_p = "SELECT fecha, DATE_FORMAT(fecha,'%Y-%m-%d'), hora, DATE_FORMAT(hora, '%H:%i:%s'), P1/1000,P2/1000, calidad FROM datos WHERE cliente = "+str(j)+" and (((fecha>'"+str(date_ini.year)+"-"+str(date_ini.month)+"-"+str(date_ini.day)+"') or (fecha='"+str(date_ini.year)+"-"+str(date_ini.month)+"-"+str(date_ini.day)+"' and hora>='"+str(date_ini.hour)+":"+str(date_ini.minute)+":00')) and ((fecha<'"+str(date_fin.year)+"-"+str(date_fin.month)+"-"+str(date_fin.day)+"') or (fecha='"+str(date_fin.year)+"-"+str(date_fin.month)+"-"+str(date_fin.day)+"' and hora<='"+str(date_fin.hour)+":"+str(date_fin.minute)+":00')))"+" UNION SELECT fecha, DATE_FORMAT(fecha,'%Y-%m-%d'), hora, DATE_FORMAT(hora, '%H:%i:%s'), P,P, calidad FROM tramas WHERE cliente = "+str(j)+" and (((fecha>'"+str(date_ini.year)+"-"+str(date_ini.month)+"-"+str(date_ini.day)+"') or (fecha='"+str(date_ini.year)+"-"+str(date_ini.month)+"-"+str(date_ini.day)+"' and hora>='"+str(date_ini.hour)+":"+str(date_ini.minute)+":00')) and ((fecha<'"+str(date_fin.year)+"-"+str(date_fin.month)+"-"+str(date_fin.day)+"') or (fecha='"+str(date_fin.year)+"-"+str(date_fin.month)+"-"+str(date_fin.day)+"' and hora<='"+str(date_fin.hour)+":"+str(date_fin.minute)+":00')))" 
        db = MySQLdb.connect(host, user,passw,bd_nombre)
        db_cursor = db.cursor()
        db_cursor.execute(datos_p)
        data = db_cursor.fetchall()

        FP=[]
        P=[]
        calidad=[]

        for dato in data:
            try:
                if dato[6] == 1511:
                    P.append(dato[5])
                if dato[6] == 1512:
                    P.append(dato[4])
                if dato[6] < 1511:
                    P.append(float(dato[4]+dato[5])/2.)

                FP.append(datetime.datetime(int(dato[1][0:4]),int(dato[1][5:7]),int(dato[1][8:10]),int(dato[3][0:2]),int(dato[3][3:5])))
                calidad.append(dato[6])
            except:
                pass
        try:
            cc = np.where(np.array(calidad) != 151)[0]
            FP = np.array(FP)[cc]
            P = np.array(P)[cc]
            M[str(j)]=pd.Series(map(float,P),FP)
        except:
            pass

    fig=plt.figure(2,edgecolor='w',facecolor='w',figsize=(100,100))
    plt.close('all')  
    plt.plot (M[str(p[0][0])].index,M[str(p[0][0])].values,color='r',label='Estacion '+str(p[0][0]))
    try:
        plt.fill_between(M[str(p[0][0])].index,M[str(p[0][0])].values,color='r',alpha=0.2)
    except:
        pass
    plt.plot (M[str(p[0][1])].index,M[str(p[0][1])].values,color='c',label='Estacion '+str(p[0][1]))
    try:
        plt.fill_between(M[str(p[0][1])].index,M[str(p[0][1])].values,color='r',alpha=0.2)
    except:
        pass
    plt.legend(loc=8, ncol=1, prop={'size':11}) #loc='upper center', bbox_to_anchor=(0.3, 0.1)

    plt.grid()  
    plt.gca().invert_yaxis()
    plt.ylabel("P (mm)")
    plt.xticks(rotation='vertical')
    plt.twinx()
    plt.gca().xaxis.set_major_formatter(matplotlib.dates.DateFormatter("%H:%M"))
    plt.plot(list(N.index),list(N.values), color='k')
    plt.xlabel("Hora")
    plt.ylabel("N (cm)")
    plt.title('Evento '+str(i[0])+' ('+str(w+1)+')')

    a = plt.axes([0.5, 0.5, .30, .35])
    plt.plot (M[str(p[0][0])].index,M[str(p[0][0])].values,color='r')
    try:
        plt.fill_between(M[str(p[0][0])].index,M[str(p[0][0])].values,color='r',alpha=0.2)
    except:
        pass  
    plt.plot (M[str(p[0][1])].index,M[str(p[0][1])].values,color='c')
    try:
        plt.fill_between(M[str(p[0][1])].index,M[str(p[0][1])].values,color='r',alpha=0.2)
    except:
        pass
    plt.grid()
    plt.gca().invert_yaxis()

    a2 = a.twinx()
    a2.plot(list(N.index),list(N.values), color='k')

    if (pp+20 < len(N.index)):
        plt.setp(a2, xlim=(N.index[pp-40],N.index[pp+20]), ylim=(N.values[pp]*0.6,N.values[pp]*1.1))
    else:
        try:
            plt.setp(a2, xlim=(N.index[pp-40],N.index[len(N.index)-1]), ylim=(N.values[pp]*0.6,N.values[pp]*1.1))
        except:
            pass

    matplotlib.rcParams.update({'font.size': 10})

    try:
        a.set_xticklabels(N.index,rotation=90,fontsize=9)
    except:
        pass
    plt.gca().xaxis.set_major_formatter(matplotlib.dates.DateFormatter("%H:%M"))
    try:
        plt.savefig(path+'Hidrografas/'+str(i[0])+'*'+str(i[1])+'.png')
    except:pass

print '-----------------------------------------'
print 'Eliminar hidrográfas para filtrar'
print '-----------------------------------------'

['2017-06-16' '23:40:00']
['2017-06-26' '21:21:00']
['2017-07-25' '17:07:00']
['2017-07-26' '23:30:00']
['2017-08-25' '01:04:00']
['2017-08-25' '03:59:00']
['2017-09-10' '01:45:00']
['2017-09-11' '11:29:00']
['2017-09-13' '14:18:00']
['2017-09-19' '16:20:00']
['2017-09-22' '19:28:00']
['2017-10-10' '21:18:00']
['2017-11-18' '15:40:00']
['2017-11-18' '22:35:00']
['2017-11-19' '00:13:00']
['2017-12-06' '14:53:00']
['2017-12-12' '20:15:00']
['2017-12-31' '06:13:00']
['2018-01-04' '15:58:00']
['2018-01-04' '20:25:00']
['2018-01-09' '04:13:00']
['2018-01-09' '07:40:00']
-----------------------------------------
Eliminar hidrográfas para filtrar
-----------------------------------------


Eliminar las hidrografas que no se tendran en cuenta en la actualizacion y continuar con el proceso

### Unir corridas

In [9]:
fechas_toda=np.loadtxt(path+'fechas_1.txt',dtype=str)
nivel_todo=np.loadtxt(path+'niveles_1.txt')
a=os.listdir(path+'/Hidrografas/')
a.sort()
a = a[1::] 

fecha_t=[] ; hora_t=[]
for kk in fechas_toda:
    fecha_t.append(kk[0])
    hora_t.append(kk[1])

fecha=[] ; hora=[] ; nivel=[] ; fecha_str = []
for k in a:
    fecha.append(k.split('*')[0])
    hora.append(k.split('*')[1].split('.')[0])
    dd=np.where(np.array(fecha_t) == k.split('*')[0])[0]
    ee=np.where(np.array(hora_t) == k.split('*')[1].split('.')[0])[0]
    ff=list(set(dd).intersection(ee))
    nivel.append(float(nivel_todo[ff]))
    fecha_str.append(k.split('*')[0]+' '+k.split('*')[1].split('.')[0])

# Guardar nuevo 
np.savetxt(path+'fechas_1.txt',fecha_str,fmt="%s")
np.savetxt(path+'niveles_1.txt',nivel,fmt="%s")

os.system("sudo cp "+path+"/Hidrografas/*.png "+path+"/Hidrografas_todas/")

np.savetxt(path+'/niveles.txt',nivel_todo,fmt="%s")
np.savetxt(path+'/fechas.txt',fechas_toda,fmt="%s")

print '--------------------------------------------------------------------------------------'
print 'Revisar que los archivos niveles.txt y fechas.txt esten actualizados'
print '--------------------------------------------------------------------------------------'

--------------------------------------------------------------------------------------
Revisar que los archivos niveles.txt y fechas.txt esten actualizados
--------------------------------------------------------------------------------------


In [10]:
path

'/media/nicolas/Home/Jupyter/Esneider/modelo_crecidas/Buscador_eventos/Aguas_Frias/'

### Humedad antecedente

In [11]:
# Archivos base de eventos historicos
acum_ant_toda=np.loadtxt(path+'acum_antecedente_3dias.txt',dtype=str)
acum_rez_toda=np.loadtxt(path+'acum_rezagado.txt',dtype=str)
niveles_toda=np.loadtxt(path+'niveles.txt',dtype=str)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: UserWarning: loadtxt: Empty input file: "/media/nicolas/Home/Jupyter/Esneider/modelo_crecidas/Buscador_eventos/Aguas_Frias/acum_antecedente_3dias.txt"
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: UserWarning: loadtxt: Empty input file: "/media/nicolas/Home/Jupyter/Esneider/modelo_crecidas/Buscador_eventos/Aguas_Frias/acum_rezagado.txt"
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
print 'Eventos antes de actualizar : ' +str(len(acum_ant_toda))

Eventos antes de actualizar : 0


In [13]:
def make_colormap(seq):
    seq = [(None,) * 3, 0.0] + list(seq) + [1.0, (None,) * 3]
    cdict = {'red': [], 'green': [], 'blue': []}
    for i, item in enumerate(seq):
        if isinstance(item, float):
            r1, g1, b1 = seq[i - 1]
            r2, g2, b2 = seq[i + 1]
            cdict['red'].append([item, r1, r2])
            cdict['green'].append([item, g1, g2])
            cdict['blue'].append([item, b1, b2])
    return mcolors.LinearSegmentedColormap('CustomMap', cdict)

c = mcolors.ColorConverter().to_rgb
rvb = make_colormap([c('cyan'), c('blue'), 0.33, c('green'), c('yellow'), 0.66, c('violet'),c('red')])

# open database connection
host      ='192.168.1.74'
user      ='usrCalidad'
passw     ='aF05wnXC;'
bd_nombre ='siata'

In [14]:

# Consulta de humedad antecedente para eventos nuevos

fechas = np.loadtxt(path+'fechas_1.txt',dtype=str)
niv = np.loadtxt(path+'niveles_1.txt',dtype=str)

dates=[]
nivp=[]

for m in niv:
    nivp.append(float(m))

for i in fechas:
    dates.append(datetime.datetime(int(i[0][0:4]),int(i[0][5:7]),int(i[0][8:10]),int(i[1][0:2]),int(i[1][3:5])))

def consultaP(fecha,rezago,t):
    
    cl = []
    P = []
    cal = []
    if t=='h':
        date_ini=fecha-datetime.timedelta(minutes=rezago)
        date_fin=fecha
    if t=='d':
        date_ini=fecha-datetime.timedelta(minutes=120+rezago)
        date_fin=fecha-datetime.timedelta(minutes=120)

    datos_p = "SELECT fecha, DATE_FORMAT(fecha,'%Y-%m-%d'), hora, DATE_FORMAT(hora, '%H:%i:%s') , Cliente, P1/1000,P2/1000, calidad FROM datos WHERE cliente IN "+str(p[0])+" and (((fecha>'"+str(date_ini.year)+"-"+str(date_ini.month)+"-"+str(date_ini.day)+"') or (fecha='"+str(date_ini.year)+"-"+str(date_ini.month)+"-"+str(date_ini.day)+"' and hora>='"+str(date_ini.hour)+":"+str(date_ini.minute)+":00')) and ((fecha<'"+str(date_fin.year)+"-"+str(date_fin.month)+"-"+str(date_fin.day)+"') or (fecha='"+str(date_fin.year)+"-"+str(date_fin.month)+"-"+str(date_fin.day)+"' and hora<='"+str(date_fin.hour)+":"+str(date_fin.minute)+":00')))"+" UNION SELECT fecha, DATE_FORMAT(fecha,'%Y-%m-%d'), hora, DATE_FORMAT(hora, '%H:%i:%s') , Cliente, P,P, calidad FROM tramas WHERE cliente IN "+str(p[0])+" and (((fecha>'"+str(date_ini.year)+"-"+str(date_ini.month)+"-"+str(date_ini.day)+"') or (fecha='"+str(date_ini.year)+"-"+str(date_ini.month)+"-"+str(date_ini.day)+"' and hora>='"+str(date_ini.hour)+":"+str(date_ini.minute)+":00')) and ((fecha<'"+str(date_fin.year)+"-"+str(date_fin.month)+"-"+str(date_fin.day)+"') or (fecha='"+str(date_fin.year)+"-"+str(date_fin.month)+"-"+str(date_fin.day)+"' and hora<='"+str(date_fin.hour)+":"+str(date_fin.minute)+":00')))"
    
    db = MySQLdb.connect(host, user,passw,bd_nombre)
    db_cursor = db.cursor()
    db_cursor.execute(datos_p)
    data = db_cursor.fetchall()
    
    for dato in data:
        if dato[7] == 1:
            P.append((dato[5]+dato[6])/2)
            cl.append(dato[4])
        if dato[7] == 1511:
            P.append(dato[6])
            cl.append(dato[4])
        if dato[7] == 1512:
            P.append(dato[5])
            cl.append(dato[4])
    
    return cl,P
    
acum_max_dia = np.zeros((len(dates)))
est_max_dia = np.zeros((len(dates)))
acum_max_hor = np.zeros((len(dates)))
est_max_hor = np.zeros((len(dates)))
savetxt = np.zeros((len(dates),2))
fechas_out = []
n_evento=[]

#  0.5  1    2    3    4    5    6    7
#[720,1440,2880,4320,5760,7200,8640,10080]: 

t=4320
for f in range(len(dates)):
    n_evento.append(f+1)
    a = consultaP(dates[f],120,'h') 
    for j in p[0]:
        pp = np.where(np.array(a[0]) == int(j))[0]
        if sum(np.array(a[1])[pp]) > acum_max_hor[f]:
            acum_max_hor[f] = sum(np.array(a[1])[pp])
            est_max_hor[f] = int(j)

    aa = consultaP(dates[f],t,'d') 
    for j in p[0]:
        pp = np.where(np.array(aa[0]) == int(j))[0]
        if sum(np.array(aa[1])[pp]) > acum_max_dia[f]:
            acum_max_dia[f] = sum(np.array(aa[1])[pp])
            est_max_dia[f] = int(j)
            savetxt[f,0] = sum(np.array(aa[1])[pp])
            savetxt[f,1] = int(j)
            
np.savetxt(path+'acum_antecedente_'+str(int(t/60/24.))+'dias.txt',savetxt,fmt="%s")

### Consultas previas

In [15]:
# Archivos base de eventos historicos
acum_rez_toda=np.loadtxt(path+'acum_rezagado.txt',dtype=str)

# Archivos ya actualizado 
niveles_toda=np.loadtxt(path+'niveles.txt',dtype=str)
fechas_toda=np.loadtxt(path+'fechas.txt',dtype=str)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: UserWarning: loadtxt: Empty input file: "/media/nicolas/Home/Jupyter/Esneider/modelo_crecidas/Buscador_eventos/Aguas_Frias/acum_rezagado.txt"
  


In [16]:
# Consultas para eventos nuevos

dt_ctec = []

for x in ctec:
    dt_ctec.append(datetime.datetime(int(x[0:4]),int(x[5:7]),int(x[8:10]),int(x[11:13]),int(x[14:16])))

# open database connection
host      ='192.168.1.74'
user      ='usrCalidad'
passw     ='aF05wnXC;'
bd_nombre ='siata'

fechas=np.loadtxt(path+'fechas_1.txt',dtype=str)
niv = np.loadtxt(path+'niveles_1.txt',dtype=str)
hum_ant=np.loadtxt(path+'acum_antecedente_3dias.txt',dtype=str)

dates=[]
nivp=[]

for m in niv:
    nivp.append(float(m))

for i in fechas:
    dates.append(datetime.datetime(int(i[0][0:4]),int(i[0][5:7]),int(i[0][8:10]),int(i[1][0:2]),int(i[1][3:5])))

Estaciones="SELECT Codigo,Nombreestacion, offsetN, red  FROM estaciones WHERE codigo=("+str(n)+")"
db = MySQLdb.connect(host, user,passw,bd_nombre)
db_cursor = db.cursor()
db_cursor.execute(Estaciones)
Cod = db_cursor.fetchall()

def consultaP(fecha,rezago,est):
    
    cl = []
    P = []
    cal = []
    if est == 0:
        date_ini=fecha-datetime.timedelta(minutes=120+rezago)
        date_fin=fecha
        datos_p = "SELECT fecha, DATE_FORMAT(fecha,'%Y-%m-%d'), hora, DATE_FORMAT(hora, '%H:%i:%s') , Cliente, P1/1000,P2/1000, calidad FROM datos WHERE cliente IN "+str(p[0])+" and (((fecha>'"+str(date_ini.year)+"-"+str(date_ini.month)+"-"+str(date_ini.day)+"') or (fecha='"+str(date_ini.year)+"-"+str(date_ini.month)+"-"+str(date_ini.day)+"' and hora>='"+str(date_ini.hour)+":"+str(date_ini.minute)+":00')) and ((fecha<'"+str(date_fin.year)+"-"+str(date_fin.month)+"-"+str(date_fin.day)+"') or (fecha='"+str(date_fin.year)+"-"+str(date_fin.month)+"-"+str(date_fin.day)+"' and hora<='"+str(date_fin.hour)+":"+str(date_fin.minute)+":00')))"+" UNION SELECT fecha, DATE_FORMAT(fecha,'%Y-%m-%d'), hora, DATE_FORMAT(hora, '%H:%i:%s') , Cliente, P,P, calidad FROM tramas WHERE cliente IN "+str(p[0])+" and (((fecha>'"+str(date_ini.year)+"-"+str(date_ini.month)+"-"+str(date_ini.day)+"') or (fecha='"+str(date_ini.year)+"-"+str(date_ini.month)+"-"+str(date_ini.day)+"' and hora>='"+str(date_ini.hour)+":"+str(date_ini.minute)+":00')) and ((fecha<'"+str(date_fin.year)+"-"+str(date_fin.month)+"-"+str(date_fin.day)+"') or (fecha='"+str(date_fin.year)+"-"+str(date_fin.month)+"-"+str(date_fin.day)+"' and hora<='"+str(date_fin.hour)+":"+str(date_fin.minute)+":00')))"                                                                                                                                                                                                                                                                                                                                                                                                                                                             
    if est != 0:
        date_ini=fecha-datetime.timedelta(minutes=120+rezago)
        date_fin=fecha-datetime.timedelta(minutes=rezago)
        datos_p = "SELECT fecha, DATE_FORMAT(fecha,'%Y-%m-%d'), hora, DATE_FORMAT(hora, '%H:%i:%s') , Cliente, P1/1000,P2/1000, calidad FROM datos WHERE cliente = "+str(est)+" and (((fecha>'"+str(date_ini.year)+"-"+str(date_ini.month)+"-"+str(date_ini.day)+"') or (fecha='"+str(date_ini.year)+"-"+str(date_ini.month)+"-"+str(date_ini.day)+"' and hora>='"+str(date_ini.hour)+":"+str(date_ini.minute)+":00')) and ((fecha<'"+str(date_fin.year)+"-"+str(date_fin.month)+"-"+str(date_fin.day)+"') or (fecha='"+str(date_fin.year)+"-"+str(date_fin.month)+"-"+str(date_fin.day)+"' and hora<='"+str(date_fin.hour)+":"+str(date_fin.minute)+":00')))"+" UNION SELECT fecha, DATE_FORMAT(fecha,'%Y-%m-%d'), hora, DATE_FORMAT(hora, '%H:%i:%s') , Cliente, P,P, calidad FROM tramas WHERE cliente = "+str(est)+" and (((fecha>'"+str(date_ini.year)+"-"+str(date_ini.month)+"-"+str(date_ini.day)+"') or (fecha='"+str(date_ini.year)+"-"+str(date_ini.month)+"-"+str(date_ini.day)+"' and hora>='"+str(date_ini.hour)+":"+str(date_ini.minute)+":00')) and ((fecha<'"+str(date_fin.year)+"-"+str(date_fin.month)+"-"+str(date_fin.day)+"') or (fecha='"+str(date_fin.year)+"-"+str(date_fin.month)+"-"+str(date_fin.day)+"' and hora<='"+str(date_fin.hour)+":"+str(date_fin.minute)+":00')))"                                                                                                                                                                                                                                                                                                                                                                                                                                                         
    db = MySQLdb.connect(host, user,passw,bd_nombre)
    db_cursor = db.cursor()
    db_cursor.execute(datos_p)
    data = db_cursor.fetchall()
    
    for dato in data:
        if dato[7] == 1:
            P.append((dato[5]+dato[6])/2)
            cl.append(dato[4])
        if dato[7] == 1511:
            P.append(dato[6])
            cl.append(dato[4])
        if dato[7] == 1512:
            P.append(dato[5])
            cl.append(dato[4])
    
    return cl,P

def consultaN(fecha,rezago,est_N):

    date=fecha-datetime.timedelta(minutes=rezago)
    datos_p = "SELECT (" +str(Cod[0][2])+"-pr) FROM datos WHERE cliente = "+str(est_N)+" and fecha='"+str(date.year)+"-"+str(date.month)+"-"+str(date.day)+"' and hora>'"+str(date.hour)+":"+str(date.minute)+"' and calidad<100"                                                                                                                                                                                                                                                                                                                                                                                                                                                  
    db = MySQLdb.connect(host, user,passw,bd_nombre)
    db_cursor = db.cursor()
    db_cursor.execute(datos_p)
    try:
        data = db_cursor.fetchall()
        return data[0][0]
    except:
        pass

#---------------Agrupar por tiempo de rezago----------------------------

tiempos=range(0,35,5)
acum = np.zeros((len(dates),len(tiempos)))
inte = np.zeros((len(dates),len(tiempos)))
niva = np.zeros((len(dates),len(tiempos)))
acum_max = np.zeros((len(dates)))
est_max = np.zeros((len(dates))) ; est_max[:] = 1
fechas_out = []
n_evento=[]

for f in range(len(dates)):
    n_evento.append(f+1)
    a = consultaP(dates[f],60,0) 
    for j in p[0]:
        pp = np.where(np.array(a[0]) == int(j))[0]
        if sum(np.array(a[1])[pp]) >= acum_max[f]:
            acum_max[f] = sum(np.array(a[1])[pp])
            est_max[f] = int(j)

    for t in range(len(tiempos)):
        aa = consultaP(dates[f],tiempos[t],est_max[f])
        if len(aa[1]) > 0:
            ii = np.where(np.array(aa[1]) == max(aa[1]))[0]
            ee = np.where(np.array(p[0]) == est_max[f])[0]
            if dates[f] < np.array(dt_ctec)[ee][0]:
                inte[f,t] = (np.array(aa[1])[ii][0])*12
            if dates[f] >= np.array(dt_ctec)[ee][0]:
                inte[f,t] = sum(aa[1][ii[len(ii)-1]-2:ii[len(ii)-1]+2])*12
            acum[f,t] = sum(np.array(aa[1]))
        try:
            niva[f,t] = float(consultaN(dates[f],tiempos[t],n))
        except:
            pass
        
np.savetxt(path+'acum_rezagado.txt',acum,fmt="%s")
np.savetxt(path+'intensidad.txt',inte[::,0],fmt="%s")

In [17]:
print 'Numero de eventos final : ' +str(len(acum))+' , '+str(len(inte[::,0]))+' , '+str(len(niveles_toda))

Numero de eventos final : 15 , 15 , 15


In [26]:
# Archivos base de eventos historicos
acum_ant_toda=np.loadtxt(path+'acum_antecedente_3dias.txt',dtype=str)
acum_rez_toda=np.loadtxt(path+'acum_rezagado.txt',dtype=str)
niveles_toda=np.loadtxt(path+'niveles.txt',dtype=str)
intensidad_toda=np.loadtxt(path+'intensidad.txt',dtype=str)

# Scatter intensidad vs Acumulado para cada rezago - el color indica el nivel pico

fig=plt.figure(edgecolor='w',facecolor='w',figsize=(12,9))
ax1=fig.add_subplot(1,1,1)
plt.scatter(intensidad_toda,acum_rez_toda[:,0], c=niveles_toda, s=100, cmap = rvb) 
plt.xlabel('Intensidad $[mm/h]$',fontsize=18)
plt.ylabel('Pacumulado $[mm]$',fontsize=18)
plt.xlim(0,30)
plt.ylim(0,60)
cbar=plt.colorbar()
cbar.ax.set_ylabel('Nivel $[cm]$',size=18)
matplotlib.rcParams.update({'font.size': 18})
plt.grid(linestyle='--')
# plt.title('Label = Numero evento') # 'Label = Numero estacion'
plt.savefig(path+'IvsAvsN.png')

In [27]:
# scatter A_hora y A_dia pintando segun nivel (3)

plt.close('all')
fig=pl.figure(facecolor='w',edgecolor='w',figsize=(12,9))
ax1=fig.add_subplot(1,1,1)

ejey = [float(i) for i in acum_ant_toda[:,0]]
ejex = [float(j) for j in acum_rez_toda[:,0]]

plt.scatter(ejex,ejey, c=niveles_toda, s=100, cmap = rvb)     
plt.xlabel('Acumulado evento (mm)', size=18)
plt.ylabel(u'Acumulado 3 días (mm)', size=18)
plt.xlim(0,60)
plt.ylim(0,60)
plt.grid(linestyle='--')
cbar=plt.colorbar()
cbar.ax.set_ylabel('Nivel (cm)', size=18)
matplotlib.rcParams.update({'font.size': 18})
plt.savefig(path+'Ah_vs_Ad_3.0.png')

### Reglas

In [28]:
fechas=np.loadtxt(path+'fechas.txt',dtype=str)
hum_ant=np.genfromtxt(path+'acum_antecedente_3dias.txt',dtype=float)
acum_rez=np.genfromtxt(path+'acum_rezagado.txt',dtype=float)
nivp=np.genfromtxt(path+'niveles.txt',dtype=float)
dates=[]
mes=[]

for i in fechas:
    dates.append(datetime.datetime(int(i[0][0:4]),int(i[0][5:7]),int(i[0][8:10]),int(i[1][0:2]),int(i[1][3:5])))
    mes.append(int(i[0][5:7]))

In [29]:
#limx = [20,0] ; limy = [20,60] 

mod = 1

clases_ha = 3
clases_pe = 6
tiempos=[5,10,15,20,25,30]
hum = hum_ant[:,0]

binxa = (np.max(hum)-np.min(hum))/clases_ha
ejexa = np.arange(np.min(hum),np.max(hum)+binxa, binxa)
a = list(ejexa) ; a.append(np.max(hum))
ejexa = a

#Clasificar por humedad antecedente en tres rangos

rango_ha = np.zeros((len(hum)))

for i in range(clases_ha):

    for k in range(len(nivp)):
        if (hum[k] >= ejexa[i]) and (hum[k] <= ejexa[i+1]):
            rango_ha[k]=i+1

#Clasificar por acumulado del evento considerando humedad antecedente

rango_pe=np.zeros((len(hum),6)) #aqui estan los tiempos

binx = (np.max(acum_rez[:,:])-np.min(acum_rez[:,:]))/clases_pe
ejex = np.arange(np.min(acum_rez[:,:]),np.max(acum_rez[:,:])+binx, binx)
reglas = np.zeros((clases_pe,len(tiempos)*5))
reglas[:,:] = np.nan

for j in range(len(tiempos)):
    for i in range(clases_pe):
        for k in range(len(nivp)):
            if (acum_rez[k,j+1] >= ejex[i]) and (acum_rez[k,j+1] <= ejex[i+1]):
                rango_pe[k,j]=i+1

np.savetxt(path+'rangos_ha.txt',rango_ha,fmt="%s")
np.savetxt(path+'rangos_pe.txt',rango_pe,fmt="%s")

#------------------------------------------------------------------------ 

if mod == 2:

#Graficar percentiles

    for t in range(len(tiempos)):

        for i in range(clases_ha+1):

            int_ha = np.mean(ejexa[i:i+2])
            ii = np.where(np.array(rango_ha) == i+1)[0]
            rango_pe2 = rango_pe[ii,t]
            nivp2 = nivp[ii]
            p10=[] ; p25=[] ; p40=[] ; p50=[] ; p60=[] ; p75=[] ; p90=[]
            xplot=[]
            plt.close ('all')
            fig=plt.figure(1,edgecolor='w',facecolor='w',figsize=(10,7))

            for j in range(clases_pe):

                jj = np.where(np.array(rango_pe2) == j+1)[0]
                if len(jj)>0:					
                    nivp3 = nivp2[jj]
    #plt.scatter(np.ones((len(nivp3)))*np.mean(ejex[j:j+2]),nivp3,c='k')
                    if len(jj)>1:
                        xplot.append(np.mean(ejex[j:j+2]))
                        p10.append(np.percentile(nivp3,10))
                        p25.append(np.percentile(nivp3,25))
                        p40.append(np.percentile(nivp3,40))
                        p50.append(np.percentile(nivp3,50))
                        p60.append(np.percentile(nivp3,60))
                        p75.append(np.percentile(nivp3,75))
                        p90.append(np.percentile(nivp3,90))

            plt.plot(xplot,p10,c='c',label='Percentil 10-90')
            plt.plot(xplot,p90,c='c')
            plt.fill_between(xplot,p10, p90, color='cyan', alpha='0.5')	
            plt.plot(xplot,p25,c='g',label='Percentil 25-75')
            plt.plot(xplot,p75,c='g')
            plt.fill_between(xplot,p25, p75, color='green', alpha='0.5')
            plt.plot(xplot,p40,c='b',label='Percentil 40-60')
            plt.plot(xplot,p60,c='b')
            plt.fill_between(xplot,p40, p60, color='blue', alpha='0.5')
            plt.plot(xplot,p50,c='r',label='Percentil 50')	

            #plt.title('t=-'+str(tiempos[t])+' min Hum='+ "%.2f" % int_ha+' mm' ,fontsize=18)
            plt.xlabel('Acumulado 120 min $[mm]$',fontsize=18)
            plt.tick_params(axis='x', which='major', labelsize=14)
            plt.ylabel('Nivel pico $[cm]$',fontsize=18)
            plt.tick_params(axis='x', which='major', labelsize=14)
#             plt.xlim(0,40)
#             plt.ylim(100,220)
            plt.grid()
            plt.legend(loc=4)
            plt.savefig(path+'Scatter_t=-'+str(tiempos[t])+' min Hum='+ "%.2f" % int_ha+' mm' +'.png',bbox_inches='tight')


#------------------------------------------------------------------------

if mod == 1:

#Graficar percentiles

    for t in range(len(tiempos)):

        p10=[] ; p25=[] ; p40=[] ; p50=[] ; p60=[] ; p75=[] ; p90=[]
        xplot=[]
        plt.close ('all')
        fig=plt.figure(1,edgecolor='w',facecolor='w',figsize=(10,7))

        for j in range(clases_pe):

            jj = np.where(np.array(rango_pe[:,t]) == j+1)[0]
            if len(jj)>0:
                nivp3 = nivp[jj]
    #plt.scatter(np.ones((len(nivp3)))*np.mean(ejex[j:j+2]),nivp3,c='k')
                if len(jj)>1:
                    xplot.append(np.mean(ejex[j:j+2]))
                    p10.append(np.percentile(nivp3,10))
                    p25.append(np.percentile(nivp3,25))
                    p40.append(np.percentile(nivp3,40))
                    p50.append(np.percentile(nivp3,50))
                    p60.append(np.percentile(nivp3,60))
                    p75.append(np.percentile(nivp3,75))
                    p90.append(np.percentile(nivp3,90))

        plt.plot(xplot,p10,c='c',label='Percentil 10-90')
        plt.plot(xplot,p90,c='c')
        plt.fill_between(xplot,p10, p90, color='cyan', alpha='0.5')	
        plt.plot(xplot,p25,c='g',label='Percentil 25-75')
        plt.plot(xplot,p75,c='g')
        plt.fill_between(xplot,p25, p75, color='green', alpha='0.5')
        plt.plot(xplot,p40,c='b',label='Percentil 40-60')
        plt.plot(xplot,p60,c='b')
        plt.fill_between(xplot,p40, p60, color='blue', alpha='0.5')
        plt.plot(xplot,p50,c='r',label='Percentil 50')

        #plt.title('t=-'+str(tiempos[t])+' min',fontsize=18)
        plt.xlabel('Acumulado 120 min $[mm]$',fontsize=18)
        plt.tick_params(axis='x', which='major', labelsize=14)
        plt.ylabel('Nivel pico $[cm]$',fontsize=18)
        plt.tick_params(axis='x', which='major', labelsize=14)
#         plt.xlim(0,40)
#         plt.ylim(100,220)
        plt.grid()
        plt.legend(loc=4)
        plt.savefig(path+'Scatter_t=-'+str(tiempos[t])+' min'+'.png',bbox_inches='tight')

#-----------------------------------------------------------------------

#histograma de niveles

clases = 6
binx = (np.max(nivp)-np.min(nivp))/clases
ejex = np.arange(np.min(nivp), np.max(nivp)+binx, binx)
hist = ((np.histogram(nivp,bins=ejex))[0])
plt.close ('all')
fig=plt.figure(1,edgecolor='w',facecolor='w',figsize=(15,15))
ax1=fig.add_subplot(1,1,1)   
plt.bar(np.arange(np.min(nivp)+binx, np.max(nivp)+binx,binx)-binx/2,(hist/float(len(nivp)))*100,align='center',width=8)
plt.xlim(70,150)
ax1.set_xticks (np.arange(np.min(nivp)+binx, np.max(nivp)+binx,binx)-binx/2)
ax1.set_xticklabels (map (lambda (x) : "%.2f" % (x), np.arange(np.min(nivp)+binx, np.max(nivp)+binx,binx)-binx/2), size=12)
plt.grid()
plt.title('Numero eventos - Total = '+str(len(nivp)),size=24)
plt.xlabel('Nivel [cm]',size=24)
plt.ylabel('Frecuencia [%]',size=24)
plt.tick_params(axis='y', which='major', labelsize=22)
plt.tick_params(axis='x', which='major', labelsize=22)
plt.savefig(path+'hist_niveles.png')

#ciclo anual

p10=np.zeros((12))
p25=np.zeros((12))
p50=np.zeros((12))
p75=np.zeros((12))
p90=np.zeros((12))


for i in range(12):
    tem = []
    for j in range(len(mes)):
        if mes[j] == i+1:
            tem.append(nivp[j])
    try:
        p10[i]=np.percentile(tem,10)
        p25[i]=np.percentile(tem,25)
        p50[i]=np.percentile(tem,50)
        p75[i]=np.percentile(tem,75)
        p90[i]=np.percentile(tem,90)
    except:
        pass

plt.close ('all')
meses=['Ene','Feb','Mar','Abr','May','Jun','Jul','Ago','Sep','Oct','Nov','Dic']
fig=plt.figure(1,edgecolor='w',facecolor='w',figsize=(15,15))
ax1=fig.add_subplot(1,1,1) 
plt.plot(range(1,13,1),p10,label='Percentil 10',linewidth=2)
plt.scatter(range(1,13,1),p10,s=50,color='b')
plt.plot(range(1,13,1),p50,label='Percentil 50',linewidth=2)
plt.scatter(range(1,13,1),p50,s=50,color='g')
plt.plot(range(1,13,1),p90,label='Percentil 90',linewidth=2)
plt.scatter(range(1,13,1),p90,s=50,color='r')
plt.ylabel('Nivel [cm]',size=24)
plt.tick_params(axis='y', which='major', labelsize=22)
plt.tick_params(axis='x', which='major', labelsize=22)
plt.grid()
plt.xticks(range(1,13,1),meses,size=20)
plt.xlim(1,12)
plt.legend(loc=2)
plt.savefig(path+'ciclo_anual.png')

horas = []
for h in fechas:
    horas.append(float(h[1].split(':')[0]))

fre = np.zeros((24))
for i in horas:
    fre[int(i)] = fre[int(i)] + 1

plt.close ('all')
fig=plt.figure(1,edgecolor='w',facecolor='w',figsize=(15,15))
horas=['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24']
plt.bar(range(1,25,1),(np.array(fre)/sum(fre))*100,width=0.9)
plt.grid()
plt.xlim(1,25)
plt.xticks(np.arange(1.5,25.5,1),horas,size=16)
plt.tick_params(axis='y', which='major', labelsize=22)
plt.ylabel('Frecuencia [%]',size=24)
plt.xlabel('Horas',size=24)
plt.savefig(path+'ciclo_horario.png')

ValueError: incompatible sizes: argument 'height' must be length 7 or scalar

### Copiar archivos a folder operacional

In [30]:
os.system('cp '+path+'niveles.txt /media/nicolas/Home/Jupyter/Esneider/modelo_crecidas/files/niveles'+str(n)+'.txt')
os.system('cp '+path+'acum_antecedente_3dias.txt /media/nicolas/Home/Jupyter/Esneider/modelo_crecidas/files/acum_antecedente_3dias'+str(n)+'.txt')
os.system('cp '+path+'acum_rezagado.txt /media/nicolas/Home/Jupyter/Esneider/modelo_crecidas/files/acum_rezagado'+str(n)+'.txt')
os.system('cp '+path+'rangos_ha.txt /media/nicolas/Home/Jupyter/Esneider/modelo_crecidas/files/rangos_ha'+str(n)+'.txt')
os.system('cp '+path+'rangos_pe.txt /media/nicolas/Home/Jupyter/Esneider/modelo_crecidas/files/rangos_pe'+str(n)+'.txt')

0

NOTA: Se puede ejecutar el codigo de validacion para calcular puntaje de las predicciones historicas

NOTA: Si es estacion de transito copiar tambien fechas

In [41]:
os.system('cp '+path+'fechas.txt /media/nicolas/Home/Jupyter/Esneider/modelo_crecidas/files/fechas'+str(n)+'.txt')


0